# `SimpleRNN`
```py
model.add(SimpleRNN(
  hidden_size,
  input_shape=(timesteps, input_dim)
))

# or

model.add(SimpleRNN(
  hidden_size,
  input_length=M,
  input_dim=N
))
```
- hidden_size : 은닉 상태의 크기를 정의<br><br>
  - 메모리 셀이 다음 시점의 메모리 셀과 출력층으로 보내는 값의 크기(`output_dim`)와 동일<br><br>
  - 중소형 모델의 경우 보통 128, 256, 512, 1024 등의 값을 사용<br><br>
- `timesteps` = 입력 시퀀스의 길이(`input_length`)라고 표현하기도 함<br><br>
- `input_dim` = 입력의 크기

In [188]:
# RNN 층에 대한 코드
from keras.models import Sequential
from keras.layers import SimpleRNN

model = Sequential()

model.add(SimpleRNN(
  3, # 은닉층 노드 개수
  input_shape=(2, 10) # 입력 시퀀스의 길이, 입력의 크기
))

# (10 * 3) + (3 ** 2) + 3 = 42
#  입력층      은닉층   노드

# model.add(SimpleRNN(
#   3,
#   input_length=2,
#   input_dim=10
# ))
# 와 동일함

model.summary()

Model: "sequential_53"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_52 (SimpleRNN)   (None, 3)                 42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [189]:
from keras.models import Sequential
from keras.layers import SimpleRNN

model = Sequential()

model.add(SimpleRNN(
  3,
  batch_input_shape=(8, 2, 10) # return_sequences=false인 경우
))
# 8 : batch 크기
# 2 : 입력 시퀀스 길이
# 10 : 입력의 크기

model.summary()

Model: "sequential_54"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_53 (SimpleRNN)   (8, 3)                    42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [190]:
from keras.models import Sequential
from keras.layers import SimpleRNN

model = Sequential()

model.add(SimpleRNN(
  3,
  batch_input_shape=(8, 2, 10),
  return_sequences=True # 입력 시퀀스까지 출력
))
# 8 : batch 크기
# 2 : 입력 시퀀스 길이
# 10 : 입력의 크기

model.summary()

Model: "sequential_55"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_54 (SimpleRNN)   (8, 2, 3)                 42        
                                                                 
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


# `LSTM (Long Short-Term Memory)`

In [191]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

text = '''
경마장에 있는 말이 뛰고 있다\n 
그의 말이 법이다\n 
가는 말이 고와야 오는 말이 곱다\n
'''

t = Tokenizer()

t.fit_on_texts([text])

vocab_size = len(t.word_index) + 1

print('단어 집합의 크기 : %d' % vocab_size)
print(t.word_index) # 각 단어와 단어에 부여된 정수 인덱스 출력

단어 집합의 크기 : 12
{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [192]:
sequences = list()

for line in text.split('\n'):
  encoded = t.texts_to_sequences([line])[0]

  for i in range(1, len(encoded)):
    sequence = encoded[:i + 1]
    sequences.append(sequence)

print('학습에 사용할 샘플의 개수 : %d' % len(sequences))

학습에 사용할 샘플의 개수 : 11


In [193]:
print(sequences)
# 위의 데이터는 아직 레이블로 사용될 단어를 분리하지 않은 훈련 데이터이다.
# [2, 3]은 [경마장에, 있는]에 해당되며 [2, 3, 1]은 [경마장에, 있는, 말이]에 해당된다.
# 전체 훈련 데이터에 대해서 맨 우측에 있는 단어에 대해서만 레이블로 분리해야 한다.
# 우선 전체 샘플에 대해서 길이를 일치시켜 준다.
# 가장 긴 샘플의 길이를 기준으로 한다.
# 현재 육안으로 봤을 때, 길이가 가장 긴 샘플은 [8, 1, 9, 10, 1, 11]이고 길이는 6이다.

max_len = max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력

print('\n샘플의 최대 길이 : {}'.format(max_len))

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]

샘플의 최대 길이 : 6


In [194]:
# 전체 샘플의 길이를 6으로 패딩
sequences = pad_sequences(
  sequences,
  maxlen=max_len,
  padding='pre'
)

print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [195]:
sequences = np.array(sequences)

X = sequences[:, :-1] # 학습 데이터
y = sequences[:, -1] # 정답 데이터

print(X)
print()
print(y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]

[ 3  1  4  5  1  7  1  9 10  1 11]


In [196]:
y = to_categorical(
  y,
  num_classes=vocab_size
)
# 원-핫 인코딩 수행

print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


# `Embedding`
```py
tf.keras.layers.Embedding(
  input_dim,
  output_dim,
  embeddings_initializer='uniform',
  embeddings_regularizer=None,
  activity_regularizer=None,
  embeddings_constraint=None,
  mask_zero=False,
  input_length=None,
  **kwargs
)
```
- 인수<br><br>
  - `input_dim` : 입력 크기<br><br>
  - `output_dim` : 출력 크기<br><br>
  - `input_length` : 입력 데이터의 길이<br><br>
- 단어를 밀집 벡터로 만드는 일을 수행한다.<br><br>
정수 인코딩이 된 단어들을 입력으로 받아 수행한다.<br><br>
단어를 랜덤한 값을 가지는 밀집 벡터로 변환한 뒤에,<br><br>
인공 신경망의 가중치를 학습하는 것과 같은 방식으로 단어 벡터를 학습하는 방법을 사용한다.

In [197]:
# 모델 설계
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN
import tensorflow as tf

model = Sequential()

model.add(Embedding(
  vocab_size,
  10,
  input_length=max_len - 1
))

model.add(SimpleRNN(32))

model.add(Dense(
  vocab_size,
  activation='softmax'
))

model.summary()

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

import datetime
log_dir = 'logs\\fit\\' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(
  X,
  y, 
  epochs=200,
  verbose=2,
  callbacks=tensorboard_callback
)

Model: "sequential_56"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 5, 10)             120       
                                                                 
 simple_rnn_55 (SimpleRNN)   (None, 32)                1376      
                                                                 
 dense_14 (Dense)            (None, 12)                396       
                                                                 
Total params: 1,892
Trainable params: 1,892
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_

In [198]:
def sentence_generation(model, t, current_word, n):
  # 모델, 토크나이저, 현재 단어, 반복 횟수
  init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기 위해 저장

  sentence = ''

  for _ in range(n): # n번 반복
    encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
    encoded = pad_sequences(
      [encoded],
       maxlen=5,
       padding='pre'
    )
    # 데이터에 대한 패딩

    y_prob = model.predict(encoded, verbose=0)
    result = y_prob.argmax(axis=-1)
  
  # 입력한 X(현재 단어)에 대해 Y를 예측하고 Y(예측한 단어)fmf result에 저장
    for word, index in t.word_index.items():
      if index == result: # 만약예측한 단어와 인덱스와 동일한 단어가 있다면
        break # 해당 단어가 예측 단어이므로 끝

    current_word = current_word + ' ' + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
    sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
  
  # for문이므로 이 행동을 다시 반복
  sentence = init_word + sentence

  return sentence

print(sentence_generation(model, t, '경마장에', 4))
print(sentence_generation(model, t, '그의', 2)) # 2번 예측
print(sentence_generation(model, t, '가는', 5)) # 5번 예측

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다


# `LSTM`을 이용하여 텍스트 생성하기

In [199]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

df = pd.read_csv('./source/ArticlesApril2018.csv') # 데이터 로드

df

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319,5ae82c93068401528a2ab969,1004,By CLAIRE CAIN MILLER,article,This Common Question Reinforces the Gender Pay...,"['Discrimination', 'Wages and Salaries', 'Labo...",68,Upshot,3,2018-05-01 09:00:01,Unknown,Several states and cities have ordered employe...,The New York Times,News,https://www.nytimes.com/2018/05/01/upshot/how-...
1320,5ae82c95068401528a2ab96b,1043,By TRACY J. GATES,article,"Anna, Llama and Me","['Friendship', 'Dewdney, Anna', 'Writing and W...",65,Well,0,2018-05-01 09:00:02,Family,"The beginning, middle and end of a picture boo...",The New York Times,News,https://www.nytimes.com/2018/05/01/well/family...
1321,5ae82c9d068401528a2ab96d,659,Interview by AUDIE CORNISH,article,Gen. Michael Hayden Has One Regret: Russia,"['Classified Information and State Secrets', '...",66,Magazine,70,2018-05-01 09:00:06,Unknown,"The former N.S.A. and C.I.A. chief on Trump, S...",The New York Times,News,https://www.nytimes.com/2018/05/01/magazine/ge...
1322,5ae82c9f068401528a2ab96f,1155,By JASON ROBERT BROWN,article,There Is Nothin’ Like a Tune,"['Books and Literature', 'Purdum, Todd S', 'Th...",68,BookReview,17,2018-05-01 09:00:07,Book Review,"In “Something Wonderful,” Todd S. Purdum analy...",The New York Times,Review,https://www.nytimes.com/2018/05/01/books/revie...


In [200]:
print('열의 개수 :', len(df.columns))
print(df.columns)

열의 개수 : 15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [201]:
df['headline'].isnull().values.any()

False

In [202]:
headline = [] # 리스트 선언
headline.extend(list(df.headline.values)) # 헤드라인의 값들을 리스트로 저장
headline[:5] # 상위 5개만 출력

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [203]:
print('총 샘플의 개수 : {}'.format(len(headline))) # 현재 샘플의 개수

총 샘플의 개수 : 1324


In [204]:
headline = [n for n in headline if n != 'Unknown'] # Unknown(잡음) 값을 가진 샘플 제거
print('노이즈 값 제거 후 샘플의 개수 : {}'.format(len(headline))) # 제거 후 샘플의 개수

노이즈 값 제거 후 샘플의 개수 : 1214


In [205]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [206]:
def repreprocessing(s):
  s = s.encode('utf8').decode('ascii', 'ignore')

  return ''.join(c for c in s if c not in punctuation).lower() # 구두점 제거와 동시에 소문자화

text = [repreprocessing(x) for x in headline]

text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [207]:
t = Tokenizer()

t.fit_on_texts(text)

vocab_size = len(t.word_index) + 1

print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


In [208]:
sequences = list()

for line in text:
  encoded = t.texts_to_sequences([line])[0] # 각 샘플에 대한 정수 인코딩

  for i in range(1, len(encoded)):
    sequence = encoded[:i + 1]
    sequences.append(sequence)

sequences[:11] # 11개의 샘플 출력

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [209]:
index_to_word = {}

for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
  index_to_word[value] = key

print('빈도 수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도 수 상위 582번 단어 : offer


In [210]:
max_len=max(len(l) for l in sequences) # 가장 긴 샘플의 길이 확인
print('샘플의 최대 길이 : {}'.format(max_len))

# 가장 긴 샘플의 길이인 24로 모든 샘플의 길이를 패딩
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre') 
print(sequences[:3])

sequences = np.array(sequences)

X = sequences[:,:-1] # 학습 데이터
y = sequences[:,-1] # 정답 데이터

print(X[:3])
print(y[:3]) # 레이블 3개 출력

y = to_categorical(y, num_classes=vocab_size) # 레이블 데이터 y에 대해서 원-핫 인코딩 수행
y

샘플의 최대 길이 : 24
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]
[ 269  371 1115]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [211]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

model = Sequential()

# y 데이터를 분리하였으므로 이제 X데이터의 길이는 기존 데이터의 길이 - 1
model.add(Embedding(
  vocab_size,
  10,
  input_length=max_len - 1
))

model.add(LSTM(128))

model.add(Dense(
  vocab_size,
  activation='softmax'
))

model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

model.fit(
  X,
  y,
  epochs=200,
  verbose=2
)

Epoch 1/200
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
244/244 - 5s - loss: 7.6379 - accuracy: 0.0278 - 5s/epoch - 20ms/step
Epoch 2/200
244/244 - 4s - loss: 7.1166 - accuracy: 0.0297 - 4s/epoch - 14ms/step
Epoch 3/200
244/244 - 3s - loss: 6.9785 - accuracy: 0.0342 - 3s/epoch - 14ms/step
Epoch 4/200
244/244 - 3s - loss: 6.8551 - accuracy: 0.0410 - 3s/epoch - 14ms/step
Epoch 5/200
244/244 - 4s - loss: 6.6952 - accuracy

In [213]:
def sentence_generation(model, t, current_word, n):
  # 모델, 토크나이저, 현재 단어, 반복할 횟수
  init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장

  sentence = ''

  for _ in range(n): # n번 반복
    encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
    encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩

    y_prob = model.predict(encoded, verbose=0)
    result = y_prob.argmax(axis=-1)
    
    for word, index in t.word_index.items(): 
      if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
        break # 해당 단어가 예측 단어이므로 break

    current_word = current_word + ' '  + word
    # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
    sentence = sentence + ' ' + word
    # 예측 단어를 문장에 저장

  sentence = init_word + sentence

  return sentence

# 임의의 단어 'i'에 대해서 10개의 단어를 추가 생성
print('\n', sentence_generation(model, t, 'i', 10))

# 임의의 단어 'how'에 대해서 10개의 단어를 추가 생성
print('\n', sentence_generation(model, t, 'how', 10))


 i want to be rich and im not sorry national guard

 how to make a crossword puzzle floods schools allout may limit
